In [1]:
import urllib.request
import pandas as pd

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7f3bf5c596d8>)

In [3]:
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [4]:
print('전체 샘플의 개수 : {}'.format(len(df)))

전체 샘플의 개수 : 50000


In [5]:
train_df = df[:25000]
test_df = df[25000:]

In [6]:
train_df.to_csv("train_data.csv", index=False) # index = False : 인덱스를 저장하지 않음
test_df.to_csv("test_data.csv", index=False)

- sequential : 시퀀스 데이터 여부. (True가 기본값)
- use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
- tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
- lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
- batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부(False가 기본값)
- is_target : 레이블 데이터 여부. (False가 기본값)
- fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.


In [8]:
# 필드를 통해 앞으로 어떤 전처리를 할 것인지를 정의
from torchtext import data

TEXT = data.Field(sequential = True, 
                  use_vocab = True, 
                  tokenize = str.split, 
                  lower = True 
                  batch_first = True, 
                  fix_length = 20) 

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

In [9]:
# TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행
from torchtext.data import TabularDataset

- path : 파일이 위치한 경로.
- format : 데이터의 포맷.
- fields : 위에서 정의한 필드를 지정. 첫번째 원소는 데이터 셋 내에서 해당 필드를 호칭할 이름, 두번째 원소는 지정할 필드.
- skip_header : 데이터의 첫번째 줄은 무시.

In [10]:
train_data, test_data = TabularDataset.splits(
    path='.', train='train_data.csv', test = 'test_data.csv', format='csv',
    fields = [('text', TEXT), ('label', LABEL)], skip_header=True)

In [11]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

훈련 샘플의 개수 : 25000
테스트 샘플의 개수 : 25000


In [12]:
print(vars(train_data[0])) # vars : 인덱스 샘플 확인

{'text': ['My', 'family', 'and', 'I', 'normally', 'do', 'not', 'watch', 'local', 'movies', 'for', 'the', 'simple', 'reason', 'that', 'they', 'are', 'poorly', 'made,', 'they', 'lack', 'the', 'depth,', 'and', 'just', 'not', 'worth', 'our', 'time.<br', '/><br', '/>The', 'trailer', 'of', '"Nasaan', 'ka', 'man"', 'caught', 'my', 'attention,', 'my', 'daughter', 'in', "law's", 'and', "daughter's", 'so', 'we', 'took', 'time', 'out', 'to', 'watch', 'it', 'this', 'afternoon.', 'The', 'movie', 'exceeded', 'our', 'expectations.', 'The', 'cinematography', 'was', 'very', 'good,', 'the', 'story', 'beautiful', 'and', 'the', 'acting', 'awesome.', 'Jericho', 'Rosales', 'was', 'really', 'very', 'good,', "so's", 'Claudine', 'Barretto.', 'The', 'fact', 'that', 'I', 'despised', 'Diether', 'Ocampo', 'proves', 'he', 'was', 'effective', 'at', 'his', 'role.', 'I', 'have', 'never', 'been', 'this', 'touched,', 'moved', 'and', 'affected', 'by', 'a', 'local', 'movie', 'before.', 'Imagine', 'a', 'cynic', 'like', 'me

In [13]:
print(train_data.fields.items())

dict_items([('text', <torchtext.data.field.Field object at 0x7f3ba1b122b0>), ('label', <torchtext.data.field.Field object at 0x7f3ba1b12278>)])


- min_freq : 단어 집합에 추가 시 단어의 최소 등장 빈도 조건을 추가.
- max_size : 단어 집합의 최대 크기를 지정.

In [14]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000) # build_vocab() : 단어 집합 생성

In [16]:
# <unk>는 단어 집합에 없는 단어를 표현할 때 사용되며, <pad>는 길이를 맞추는 패딩 작업을 할 때 사용
print(TEXT.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f3ba1c02ea0>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'a': 3, 'and': 4, 'of': 5, 'to': 6, 'is': 7, 'in': 8, 'I': 9, 'that': 10, 'this': 11, 'it': 12, '/><br': 13, 'was': 14, 'as': 15, 'with': 16, 'for': 17, 'The': 18, 'but': 19, 'on': 20, 'movie': 21, 'are': 22, 'his': 23, 'film': 24, 'have': 25, 'not': 26, 'be': 27, 'you': 28, 'at': 29, 'he': 30, 'by': 31, 'an': 32, 'one': 33, 'from': 34, 'who': 35, 'like': 36, 'they': 37, 'all': 38, 'so': 39, 'just': 40, 'has': 41, 'about': 42, 'or': 43, 'her': 44, 'out': 45, 'some': 46, 'very': 47, 'more': 48, 'This': 49, 'what': 50, 'would': 51, 'when': 52, 'good': 53, 'if': 54, 'their': 55, 'only': 56, 'really': 57, 'It': 58, 'up': 59, 'even': 60, 'had': 61, 'can': 62, "it's": 63, 'which': 64, 'were': 65, 'my': 66, 'see': 67, 'no': 68, 'than': 69, 'she': 70, '-': 71, 'there': 72, 'been': 73, 'into': 74, 'get': 75, 'much': 76, 'will': 77, 'story': 78, 'because': 79, 'other': 80, 'most': 81, 'time': 82, 'me'

In [17]:
from torchtext.data import Iterator

In [18]:
batch_size = 5

train_loader = Iterator(dataset = train_data, batch_size = batch_size)
test_loader = Iterator(dataset = test_data, batch_size = batch_size)

In [19]:
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 5000
테스트 데이터의 미니 배치 수 : 5000


In [20]:
batch = next(iter(train_loader))

In [21]:
print(type(batch))

<class 'torchtext.data.batch.Batch'>


In [22]:
print(batch.text)

tensor([[ 320,   76,   41,   73,  504,   42,    2,  548,  729,    2,  453,  138,
         1982,    2, 1251,    2,    0, 9093, 4403,    2],
        [   0,    0,   34, 2936,    6,    0,    0,   18, 8732,    0, 5988,  533,
            2, 2053, 1825, 1107,    8,   21, 2157,    0],
        [   9,   25,    6, 1099,   16,  340,  409,   10,   41,    0,   13,  168,
          266,   12,  177,    3,  178, 1202,   19,  674],
        [7236,    9,  184,  260,   10,    9,  921,    0,   20,   11,  120,    9,
          220,    3,  614, 1411,    0,    4,  199,    2],
        [  16,   50,   37, 4307,  313,    4,    0,   65,   47, 2366,   39,    2,
         1039,   61,    6,  257,  391,   50,   37, 8279]])
